## Strategisthub Sales Assistance

In [4]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import os
from pathlib import Path
import bs4
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langgraph.checkpoint.memory import MemorySaver
from langchain.messages import HumanMessage
from langgraph.graph import START, END

load_dotenv()

True

In [5]:
# helper functions

def load_pdfs_from_directory(directory_path: str):
    docs = []
    for pdf_file in Path(directory_path).rglob("*.pdf"):
        try:
            # print("Reading data from: ", pdf_file)
            loader = PyPDFLoader(str(pdf_file))
            docs.extend(loader.load())
        except Exception as e:
            print(f"Error loading {pdf_file}: {e}")
    return docs

# 2. Load data from websites
def load_from_websites(urls):
    docs = []
    for url in urls:
        loader = WebBaseLoader(url)
        docs.extend(loader.load())
    return docs

In [6]:
# import weaviate
# from weaviate.classes.init import Auth
# import os

# # Best practice: store your credentials in environment variables
# weaviate_url = os.environ["WEAVIATE_URL"]
# weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=weaviate_url,
#     auth_credentials=Auth.api_key(weaviate_api_key),
# )

# print(client.is_ready())  # Should print: `True`

# client.close() 

In [7]:
# 3. Combine and split documents
pdf_docs = load_pdfs_from_directory("/home/hp/Desktop/Workplace/CustomizeGPT/data")
web_urls = ["https://strategisthub.com/services/", "https://strategisthub.com/about/", "https://strategisthub.com/case-studies/", "https://strategisthub.com/blogs/"]
web_docs = load_from_websites(web_urls)

all_docs = pdf_docs + web_docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(all_docs)

In [10]:
# 4. Create vector store
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(doc_splits, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [6]:
# Check the source or metadata of the content
def retrieve_documents(query: str):
    """Retrieve relevant documents."""
    docs = retriever.invoke(query)
    serialized = "\\n\\n".join(
        f"Source: {doc.metadata}\\nContent: {doc.page_content}"
        for doc in docs
    )
    return serialized, docs

ser,response = retrieve_documents("what are the major services of strategisthub?")

print(response[-1].metadata)
print(response[-1].page_content)

{'producer': 'iLovePDF', 'creator': 'Adobe Illustrator 26.0 (Windows)', 'creationdate': '2025-06-11T15:27:10+06:00', 'title': 'PitchDeck 2.0', 'moddate': '2025-06-11T10:33:07+00:00', 'source': '/home/hp/Desktop/Workplace/CustomizeGPT/data/Sales/Profiles & Portfolio Documents/Pitch Deck/PitchDeck.pdf', 'total_pages': 18, 'page': 1, 'page_label': '2'}
StrategistHub is a trusted technology partner that helps startups, 
SMBs, and enterprises  build scalable, high-performance digital 
solutions with the expertise of top-tier software engineers. With 
over a decade of experience in software development and
engineering services, we provide businesses with highly skilled 
engineers who bring deep technical expertise in AI-driven
solutions, custom software development, and advanced system 
architecture. Our team enables organizations to streamline


In [7]:
# # Testing crossEncoder...
# from sentence_transformers import CrossEncoder
# cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# def retrieve_documents(query: str, top_k: int = 2):
#     """Retrieve relevant documents and rerank them using cross-encoder."""
#     # Step 1: retrieve documents
#     docs = retriever.invoke(query)  # assuming this returns a list of Document objects
    
#     # Step 2: rerank using cross-encoder
#     passages = [doc.page_content if hasattr(doc, "page_content") else str(doc) for doc in docs]
#     pairs = [(query, p) for p in passages]
#     scores = cross_encoder_model.predict(pairs)
#     reranked_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
    
#     # Step 3: serialize top passages
#     serialized = "\n\n".join(
#         f"Source: {doc.metadata}\nContent: {doc.page_content}" for doc in reranked_docs[:top_k]
#     )
    
#     return serialized, reranked_docs[:top_k], scores

# ser,response, scores = retrieve_documents("what are the major services of strategisthub?")

# # print(response[-1].metadata)
# # print(response[-1].page_content)

# print(response)
# print("\n\n")
# print(ser)
# print("\n\n")
# print(scores)


In [8]:
# tool node
# 5. Create retriever tool
@tool(response_format="content_and_artifact")
def retrieve_documents(query: str):
    """Retrieve relevant documents."""
    docs = retriever.invoke(query)
    serialized = "\\n\\n".join(
        f"Source: {doc.metadata}\\nContent: {doc.page_content}"
        for doc in docs
    )
    return serialized, docs

tools = [retrieve_documents]
tool_node = ToolNode(tools)

In [9]:
# 6. Enhanced system prompt for email responses
EMAIL_SYSTEM_PROMPT = """
You are a professional email assistant for our company's sales team. Your role is to respond to customer inquiries using ONLY information from our knowledge base.

CRITICAL RULES:
1. You MUST respond in proper business email format with subject line, salutation, body, and signature
2. If the customer's question can be answered using the provided context, write a helpful, professional email response
3. If the information is NOT in the knowledge base (context shows "NO_RELEVANT_INFORMATION_FOUND"), respond with a polite email explaining this
4. Never invent information or use external knowledge
5. Maintain a professional, helpful tone in all communications
6. Format your response as a ready-to-send email
7. Always start the subject with "Re: " followed by the original subject or an appropriate title

EMAIL FORMAT:
Subject: Re: [Original Subject or Appropriate Title]

Dear [Customer Name],

[Professional email body acknowledging their query and providing information or explaining limitations]

[Clear next steps or contact information if needed]

Best regards,
[Sale Team]
[Strategisthub]
"""

In [10]:
# 6. Build LangGraph nodes
model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

def call_model(state: MessagesState):
    response = model.invoke(
        [
            SystemMessage(
                        content=EMAIL_SYSTEM_PROMPT
                    )
        ]
        + state["messages"]
        )
    return {"messages": [response]}

def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

# 7. Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# app = workflow.compile()
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [11]:
# from IPython.display import Image, display
# display(Image(app.get_graph().draw_mermaid_png()))

In [12]:
query = """
Hi, I am
Interested in finding how ai can help me as a director, contracts manager estimator and pretty much everything else to run our electrical contracting business.
"""

In [13]:
# memory = MemorySaver()
# app = workflow.compile(checkpointer=memory)

# Use with thread
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: How AI Can Help in Electrical Contracting Business

Dear [Customer Name],

Thank you for your inquiry about how AI can assist you in your roles as a director, contracts manager, estimator, and more within your electrical contracting business.

AI can provide high-impact solutions that enhance various aspects of your operations. For instance, it can significantly reduce the time required to generate first-draft ideas, allowing you to move from brief to concept in under five minutes. This acceleration in ideation can help streamline your project planning and execution.

Moreover, AI-generated ideas can be closely aligned with your business strategies, ensuring that the outputs are relevant and effective. By utilizing interactive trend modules and insights, you can encourage deeper exploration and engagement within your team, ultimately leading to improved project outcomes.

If you would like to explore specific AI tools or solutions tailored to your business needs, please le

In [15]:
print(response)

{'messages': [HumanMessage(content='\nHi, I am\nInterested in finding how ai can help me as a director, contracts manager estimator and pretty much everything else to run our electrical contracting business.\n', additional_kwargs={}, response_metadata={}, id='2e21597e-970f-4cc8-9748-b031aedfb637'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 293, 'total_tokens': 312, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_aa07c96156', 'id': 'chatcmpl-CjQ0bvMQOrOUsC3Qh6n2CekYZgxiT', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--5a095c39-0ebc-4f39-b225-2e67614430b6-0', tool_calls=[{'name': 'retrieve_documents', 'arg

In [89]:
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": "what are the major services of strategisthub?"}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Major Services of StrategistHub

Dear [Customer Name],

Thank you for your inquiry regarding the major services offered by StrategistHub.

At StrategistHub, we pride ourselves on being a trusted technology partner for startups, small to medium-sized businesses (SMBs), and enterprises. Our primary services include:

1. **Custom Software Development**: We specialize in building scalable, high-performance digital solutions tailored to meet the unique needs of our clients.

2. **AI-Driven Solutions**: Our expertise in artificial intelligence allows us to develop innovative solutions that enhance operational efficiency and user experiences.

3. **Advanced System Architecture**: We provide advanced system architecture services to ensure that our clients' technology infrastructure is robust and future-ready.

4. **Engineering Excellence**: With over a decade of experience, we deliver engineering excellence that drives digital transformation and long-term success for businesses.



In [ ]:
# """
# Strategisthub offers a range of services designed to enhance business performance and drive growth.

#  Our major services include:\n\n1. 
 
#  **Consulting Services:**\n   - Strategic planning and execution\n   - Market analysis and competitive intelligence\n   - Operational efficiency improvement\n\n2. 
 
#  **Training and Development:**\n   - Customized training programs for teams\n   - Leadership development workshops\n   - Skills enhancement sessions\n\n3. 
 
#  **Technology Solutions:**\n   - Implementation of business management software\n   - Data analytics and reporting tools\n   - IT support and infrastructure development\n\n4. 
 
#  **Marketing Services:**\n   - Digital marketing strategies\n   - Brand development and management\n   - Content creation and social media management\n\n5. 
 
#  **Financial Advisory:**\n   - Financial planning and analysis\n   - Investment strategy development\n   - Risk management solutions\n\n
 
#  These services are tailored to meet the unique needs of our clients, ensuring they receive the support necessary to achieve their business objectives. 


# If you need more specific information or details about a particular service, feel free to ask!

# """

In [ ]:
# """
# "Strategisthub offers a range of services designed to empower businesses and enhance their strategic capabilities. 
# Key services include:\n\n1. 
#
# **Consulting Services:**\n   - Strategic Planning\n   - Market Research and Analysis\n   - Business Development Strategies\n\n2. 
# **Training and Workshops:**\n   - Leadership Development\n   - Sales Training\n   - Team Building and Collaboration\n\n3. 
# **Technology Solutions:**\n   - Data Analytics and Business Intelligence\n   - Custom Software Development\n   - Digital Transformation Services\n\n4. 
# **Marketing Services:**\n   - Brand Strategy and Positioning\n   - Digital Marketing Solutions\n   - Content Creation and Management\n\n5. 
# **Project Management:**\n   - Project Planning and Execution\n   - Risk Management\n   - Performance Monitoring and Reporting\n\n
# 
# These services are tailored to meet the unique needs of each client, ensuring they receive the support necessary to achieve their strategic goals.
# If you need more specific information or details about a particular service, please let me know!"
# 
# """

In [ ]:
# """
# Strategisthub offers a range of services designed to empower businesses and enhance their strategic capabilities. 
# Our major services include:\n\n1. 
# **Consulting Services:**\n   - Strategic Planning\n   - Market Research and Analysis\n   - Business Process Optimization\n\n2. 
# **Technology Solutions:**\n   - Custom Software Development\n   - Data Analytics and Business Intelligence\n   - Cloud Solutions and Integration\n\n3. 
# **Training and Development:**\n   - Leadership Development Programs\n   - Sales and Marketing Training\n   - Workshops on Emerging Technologies\n\n4. 
# **Project Management:**\n   - End-to-End Project Management Services\n   - Agile and Waterfall Methodologies\n   - Risk Management and Mitigation Strategies\n\n5. 
# **Marketing Services:**\n   - Digital Marketing Strategy\n   - Content Creation and Management\n   - Brand Development and Positioning\n\n

# These services are tailored to meet the unique needs of our clients, ensuring they achieve their strategic goals effectively and efficiently. 

# If you need more specific information or details about any of these services, feel free to ask!"
# """

In [ ]:
# """
# Strategisthub offers a range of services designed to enhance business performance and drive growth. 
# Our major services include:\n\n1. 
#
# **Strategic Consulting:** Tailored strategies to help businesses identify opportunities, optimize operations, and achieve their goals.\n\n2. 
# **Market Research:** In-depth analysis of market trends, consumer behavior, and competitive landscapes to inform decision-making.\n\n3. 
# **Digital Marketing Solutions:** Comprehensive digital marketing strategies, including SEO, content marketing, social media management, and PPC advertising.\n\n4. 
# **Sales Enablement:** Tools and training designed to empower sales teams, improve performance, and enhance customer engagement.\n\n5. 
# **Training and Development:** Customized training programs to develop skills and knowledge across various business functions.\n\n6. 
# **Technology Solutions:** Implementation of innovative technology solutions to streamline processes and improve efficiency.\n\n
# 
# If you need more specific information about any of these services, please let me know!
# """

In [43]:
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": "what is the vision and mission of strategisthub?"}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Vision and Mission of StrategistHub

Dear [Customer Name],

Thank you for your inquiry regarding the vision and mission of StrategistHub.

**Vision**: At StrategistHub, we envision a future where businesses thrive through intelligent innovation. Our goal is to become the trusted global partner for AI-driven digital transformation, helping businesses unlock their full potential, lead markets, and build scalable impact in a rapidly evolving tech landscape.

**Mission**: Our mission is to transform complex challenges into scalable opportunities through AI-powered strategies and deep technical expertise. We aim to empower organizations of all sizes to drive transformative growth and set new standards of excellence in their respective industries.

If you have any further questions or need additional information, please feel free to reach out.

Best regards,  
Sales Team  
Strategisthub


In [41]:
# from langchain.messages import HumanMessage
# result = app.invoke({
#     "messages": [HumanMessage(content=user_input)]
# })
# print(result["messages"][-1].content)

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("diwank/dfe-base-en-1")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
embeddings = model.encode(sentences, task="dialog")

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [4, 4]

torch.Size([4, 4])
